<a href="https://colab.research.google.com/github/LudoCatt/FakeNewsDetection/blob/main/Emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis using Flair

largely inspired by [this tutorial](https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c) and [this tutorial](https://www.section.io/engineering-education/how-to-create-nlp-application-with-flair/#splitting-the-dataset) 

## load dataset from [Sem-Eval 2018](https://www.scielo.org.mx/pdf/cys/v24n3/1405-5546-cys-24-03-1159.pdf) competition

### install libraries

In [ ]:
#!pip uninstall protobuf

In [ ]:
!pip install protobuf==3.20.3 --quiet
!pip install flair --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 34.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 57.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

### import flair NLP library; visualisation of pretrained classifier

In [ ]:
import flair
# 'en-sentiment' text classifier is a pre-trained model that classifies text 
# into two emotions, with a confidence score
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2023-05-23 12:31:00,331 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpictlw8cg


100%|██████████| 253M/253M [00:12<00:00, 21.7MB/s]

2023-05-23 12:31:13,010 copying /tmp/tmpictlw8cg to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-05-23 12:31:13,542 removing temp file /tmp/tmpictlw8cg


In [ ]:
sentence = "green apples make me angry"
s = flair.data.Sentence(sentence)
print(s)

Sentence[5]: "green apples make me angry"


In [ ]:
flair_sentiment.predict(s)
total_sentiment = s.labels
print(total_sentiment)

['Sentence[5]: "green apples make me angry"'/'NEGATIVE' (0.9969)]


### We create our own model based on a dataset with multi-label sentiment classification

In [ ]:
from datasets import list_datasets, load_dataset
from pprint import pprint
import pandas as pd

 we import a dataset from the Sem-Eval 2018 competition, containing roughly 11k tweets in English, each labeled with one of 11 emotions 

In [ ]:
dataset_train = load_dataset("sem_eval_2018_task_1", "subtask5.english", split="train")
dataset_val = load_dataset("sem_eval_2018_task_1", "subtask5.english", split="validation")
dataset_test = load_dataset("sem_eval_2018_task_1", "subtask5.english", split="test")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

Dataset sem_eval_2018_task_1 downloaded and prepared to /root/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.english/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182. Subsequent calls will reuse this data.


In [ ]:
dataset_train.set_format(type="pandas")
dataset_val.set_format(type="pandas")
dataset_test.set_format(type="pandas")

In [ ]:
# view the columns of the dataset
dataset_train

Dataset({
    features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
    num_rows: 6838
})

In [ ]:
dataset_train['anger'].head()

0    False
1    False
2     True
3    False
4     True
Name: anger, dtype: bool

### preprocess the data to feed it to model

In [ ]:
df_train = pd.DataFrame(dataset_train[:])
df_val = pd.DataFrame(dataset_val[:])
df_test = pd.DataFrame(dataset_test[:])

In [ ]:
# example from the dataset
print(f"text: {df_train.iloc[5]['Tweet']}")
df_train.iloc[5]

text: No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu


ID                                                  2017-En-22190
Tweet           No but that's so cute. Atsu was probably shy a...
anger                                                       False
anticipation                                                False
disgust                                                     False
fear                                                        False
joy                                                          True
love                                                        False
optimism                                                    False
pessimism                                                   False
sadness                                                     False
surprise                                                    False
trust                                                       False
Name: 5, dtype: object

In [ ]:
emotions = ["anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", 
            "pessimism", "sadness", "surprise", "trust"]

# create a single "label" column for each dataset
df_train["label"] = df_train[emotions].idxmax(axis=1)
labeled_df_train = df_train.drop(emotions, axis=1)

df_test["label"] = df_test[emotions].idxmax(axis=1)
labeled_df_test = df_test.drop(emotions, axis=1)

df_val["label"] = df_val[emotions].idxmax(axis=1)
labeled_df_val = df_val.drop(emotions, axis=1)


In [ ]:
# format the columns to feed into model
labeled_df_train['label'] = '__label__' + labeled_df_train['label'].astype(str)
labeled_df_test['label'] = '__label__' + labeled_df_test['label'].astype(str)
labeled_df_val['label'] = '__label__' + labeled_df_val['label'].astype(str)

df_train = labeled_df_train.rename(columns={"Tweet": "text"}).drop("ID", axis=1)
df_test = labeled_df_test.rename(columns={"Tweet": "text"}).drop("ID", axis=1)
df_val = labeled_df_val.rename(columns={"Tweet": "text"}).drop("ID", axis=1)

In [ ]:
df_train.head()

,text,label
0,“Worry is a down payment on a problem you may ...,__label__anticipation
1,Whatever you decide to do make sure it makes y...,__label__joy
2,@Max_Kellerman it also helps that the majorit...,__label__anger
3,Accept the challenges so that you can literall...,__label__joy
4,My roommate: it's okay that we can't spell bec...,__label__anger


## Model training on multilabel dataset

In [ ]:
!mkdir -p data_fst

In [ ]:
df_train.to_csv("data_fst/train.csv")
df_test.to_csv("data_fst/test.csv")
df_val.to_csv("data_fst/dev.csv")

In [ ]:
from flair.data import Corpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from datasets import Dataset
from flair.datasets import CSVClassificationCorpus

In [ ]:
# load the datasets
data_folder = 'data_fst/'
column_name_map = {2:"label_topic",1:"text"}
# create a corpus class instance from our data to then feed into the model
corpus_csv = CSVClassificationCorpus(data_folder, 
                                     column_name_map=column_name_map,
                                     skip_header=True,
                                     delimiter=',',
                                     label_type='question_class')

2023-05-23 12:32:06,081 Reading data from data_fst
2023-05-23 12:32:06,082 Train: data_fst/train.csv
2023-05-23 12:32:06,083 Dev: data_fst/dev.csv
2023-05-23 12:32:06,084 Test: data_fst/test.csv


In [ ]:
# create a label dictionary from our data
label_dict_csv = corpus_csv.make_label_dictionary(label_type='question_class')
print(label_dict_csv)

2023-05-23 12:32:06,132 Computing label dictionary. Progress:


6838it [00:03, 2001.31it/s]

2023-05-23 12:32:09,561 Dictionary created for label 'question_class' with 11 values: __label__anger (seen 2748 times), __label__joy (seen 1660 times), __label__anticipation (seen 811 times), __label__fear (seen 569 times), __label__disgust (seen 452 times), __label__pessimism (seen 194 times), __label__optimism (seen 180 times), __label__sadness (seen 180 times), __label__love (seen 27 times), __label__surprise (seen 17 times)
Dictionary with 11 tags: <unk>, __label__anger, __label__joy, __label__anticipation, __label__fear, __label__disgust, __label__pessimism, __label__optimism, __label__sadness, __label__love, __label__surprise


In [ ]:
# Word embeddings provide methods that combine words and documents in different ways.
word_embeddings = [FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
# DocumentEmbeddings which embed an entire text or sentence
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)


2023-05-23 12:32:10,227 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmphz8pxr83


100%|██████████| 18.8M/18.8M [00:01<00:00, 10.1MB/s]

2023-05-23 12:32:12,587 copying /tmp/tmphz8pxr83 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2023-05-23 12:32:12,607 removing temp file /tmp/tmphz8pxr83


2023-05-23 12:32:14,004 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpt2cxredy


100%|██████████| 18.8M/18.8M [00:01<00:00, 10.7MB/s]

2023-05-23 12:32:16,234 copying /tmp/tmpt2cxredy to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2023-05-23 12:32:16,251 removing temp file /tmp/tmpt2cxredy


In [ ]:
# we define the classifier model by creating a TextClassifier instance
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict_csv,
                            label_type='question_class', multi_label=True)
trainer = ModelTrainer(classifier, corpus_csv)

In [ ]:
# train the model using Adam optimizer
import torch
trainer.train('data_fst/',
              max_epochs=50,
              optimizer=torch.optim.Adam,
              learning_rate=1.0e-3,
              mini_batch_size=16, 
              )

2023-05-23 12:32:16,343 ----------------------------------------------------------------------------------------------------
2023-05-23 12:32:16,345 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
        )
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2048, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=11, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=

100%|██████████| 56/56 [00:05<00:00, 10.74it/s]

2023-05-23 12:32:56,252 Evaluating as a multi-label problem: False
2023-05-23 12:32:56,269 DEV : loss 2.39920711517334 - f1-score (micro avg)  0.381


2023-05-23 12:32:56,553 BAD EPOCHS (no improvement): 0
2023-05-23 12:32:56,555 saving best model
2023-05-23 12:32:56,629 ----------------------------------------------------------------------------------------------------
2023-05-23 12:32:59,591 epoch 2 - iter 42/428 - loss 2.39651133 - time (sec): 2.96 - samples/sec: 226.95 - lr: 0.001000
2023-05-23 12:33:02,591 epoch 2 - iter 84/428 - loss 2.37764266 - time (sec): 5.96 - samples/sec: 225.48 - lr: 0.001000
2023-05-23 12:33:06,267 epoch 2 - iter 126/428 - loss 2.35983649 - time (sec): 9.64 - samples/sec: 209.20 - lr: 0.001000
2023-05-23 12:33:09,692 epoch 2 - iter 168/428 - loss 2.36651862 - time (sec): 13.06 - samples/sec: 205.80 - lr: 0.001000
2023-05-23 12:33:12,734 epoch 2 - iter 210/428 - loss 2.37481027 - time (sec): 16.10 - samples/sec: 208.65 - lr: 0.001000
2023-05-23 12:33:15,740 epoch 2 - iter 252/428 - loss 2.36979537 - time (sec): 19.11 - samples/sec: 211.00 - lr: 0.001000
2023-05-23 12:33:19,410 epoch 2 - iter 294/428 - lo

100%|██████████| 56/56 [00:05<00:00, 10.07it/s]

...

2023-05-23 12:50:57,360 DEV : loss 2.3183412551879883 - f1-score (micro avg)  0.5506
2023-05-23 12:50:57,869 Epoch    29: reducing learning rate of group 0 to 6.2500e-05.
2023-05-23 12:50:57,875 BAD EPOCHS (no improvement): 4
2023-05-23 12:50:57,877 ----------------------------------------------------------------------------------------------------
2023-05-23 12:50:57,880 ----------------------------------------------------------------------------------------------------
2023-05-23 12:50:57,882 learning rate too small - quitting training!
2023-05-23 12:50:57,884 ----------------------------------------------------------------------------------------------------
2023-05-23 12:50:57,982 ----------------------------------------------------------------------------------------------------


100%|██████████| 204/204 [00:13<00:00, 15.12it/s]

2023-05-23 12:51:11,723 Evaluating as a multi-label problem: True


2023-05-23 12:51:11,851 0.6993	0.4931	0.5784	0.4863
2023-05-23 12:51:11,854 
Results:
- F-score (micro) 0.5784
- F-score (macro) 0.2245
- Accuracy 0.4863

By class:
                       precision    recall  f1-score   support

       __label__anger     0.7193    0.6624    0.6897      1176
         __label__joy     0.7397    0.6483    0.6910      1052
__label__anticipation     0.4079    0.0914    0.1494       339
        __label__fear     0.6357    0.3787    0.4747       235
     __label__disgust     0.2500    0.0102    0.0196       196
   __label__pessimism     0.5000    0.1860    0.2712        86
     __label__sadness     0.0000    0.0000    0.0000        86
    __label__optimism     0.3636    0.1143    0.1739        70
        __label__love     0.0000    0.0000    0.0000        14
    __label__surprise     0.0000    0.0000    0.0000         4
       __label__trust     0.0000    0.0000    0.0000         1

            micro avg     0.6993    0.4931    0.5784      3259
            ma

{'test_score': 0.5783696238977866,
 'dev_score_history': [0.381020848310568,
  0.45170660856935363,
  0.48600883652430044,
  0.44588045234248785,
  0.5292479108635098,
  0.5253648366921474,
  0.5257142857142858,
  0.5370629370629371,
  0.514822848879248,
  0.5515358361774745,
  0.5651006711409395,
  0.5469277515192439,
  0.5448323066392882,
  0.5620827770360481,
  0.546831955922865,
  0.5354439091534756,
  0.5685752330226365,
  0.5674121405750798,
  0.5510071474983755,
  0.5460526315789475,
  0.5653896961690885,
  0.5682560418027434,
  0.5605670103092784,
  0.5553372626064178,
  0.5563517915309446,
  0.5618556701030929,
  0.5588615782664942,
  0.5519897304236201,
  0.5505761843790012],
 'train_loss_history': [2.6792705183874346,
  2.335255660240885,
  2.1629621679234066,
  2.092928094799737,
  2.003150710239505,
  1.970020895291574,
  1.9144010635871362,
  1.8842396075072154,
  1.8410915942107002,
  1.8239836784858179,
  1.7693277292343634,
  1.7548125390596436,
  1.7125222162044198,
 

## making predictions

In [ ]:
new_clf = TextClassifier.load('data_fst/best-model.pt')
from flair.data import Sentence


In [ ]:
s1 = Sentence("I just finished watching Shrek for the third time! It's the small things in life...")

In [ ]:
s2 = Sentence("I'm really looking forward to moving to Zurich! Can't wait to see some old friends")

In [ ]:
s3 = Sentence("Bruh the current president is a clown")

In [ ]:
s4 = Sentence("With things as they currently stand, I'm afraid I don't see a future for our kids")

In [ ]:
sentences = [s1, s2, s3, s4]

In [ ]:
for s in sentences:
  new_clf.predict(s)
  print(s.labels)

['Sentence[18]: "I just finished watching Shrek for the third time! It's the small things in life..."'/'__label__joy' (0.6652)]
['Sentence[18]: "I'm really looking forward to moving to Zurich! Can't wait to see some old friends"'/'__label__anticipation' (0.6501)]
['Sentence[7]: "Bruh the current president is a clown"'/'__label__anger' (0.7535)]
['Sentence[19]: "With things as they currently stand, I'm afraid I don't see a future for our kids"'/'__label__fear' (0.7026)]
